In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType

In [2]:
spark = SparkSession.builder.appName("SalesAnalytics").getOrCreate()

## Data Preparation

In [3]:
schema = StructType([
    StructField("Order ID", StringType(), True),
    StructField("Product", StringType(), True),
    StructField("Qty Ordered", StringType(), True),
    StructField("Price Each", StringType(), True),
    StructField("Order Date", StringType(), True),
    StructField("Purchase Address", StringType(), True)
])

In [4]:
sales_data_fpath = [
    "./data/salesdata/Sales_January_2019.csv",
    "./data/salesdata/Sales_February_2019.csv",
    "./data/salesdata/Sales_March_2019.csv",
    "./data/salesdata/Sales_April_2019.csv",
    "./data/salesdata/Sales_May_2019.csv",
    "./data/salesdata/Sales_June_2019.csv",
    "./data/salesdata/Sales_July_2019.csv",
    "./data/salesdata/Sales_August_2019.csv",
    "./data/salesdata/Sales_September_2019.csv",
    "./data/salesdata/Sales_October_2019.csv",
    "./data/salesdata/Sales_November_2019.csv",
    "./data/salesdata/Sales_December_2019.csv"]
sales_raw_df = spark.read.csv(sales_data_fpath, header = True, schema = schema)

In [5]:
sales_raw_df.show(10)

+--------+--------------------+-----------+----------+--------------+--------------------+
|Order ID|             Product|Qty Ordered|Price Each|    Order Date|    Purchase Address|
+--------+--------------------+-----------+----------+--------------+--------------------+
|  295665|  Macbook Pro Laptop|          1|      1700|12/30/19 00:01|136 Church St, Ne...|
|  295666|  LG Washing Machine|          1|     600.0|12/29/19 07:03|562 2nd St, New Y...|
|  295667|USB-C Charging Cable|          1|     11.95|12/12/19 18:21|277 Main St, New ...|
|  295668|    27in FHD Monitor|          1|    149.99|12/22/19 15:13|410 6th St, San F...|
|  295669|USB-C Charging Cable|          1|     11.95|12/18/19 12:38|43 Hill St, Atlan...|
|  295670|AA Batteries (4-p...|          1|      3.84|12/31/19 22:58|200 Jefferson St,...|
|  295671|USB-C Charging Cable|          1|     11.95|12/16/19 15:10|928 12th St, Port...|
|  295672|USB-C Charging Cable|          2|     11.95|12/13/19 09:29|813 Hickory St, D...|

In [6]:
sales_raw_df.printSchema()

root
 |-- Order ID: string (nullable = true)
 |-- Product: string (nullable = true)
 |-- Qty Ordered: string (nullable = true)
 |-- Price Each: string (nullable = true)
 |-- Order Date: string (nullable = true)
 |-- Purchase Address: string (nullable = true)

